In [1]:
import pandas as pd
from tab_dataset import Cdataset
import ntv_pandas as npd

In [23]:
payload1 = pd.read_json("static_14h12m44s.json", orient="record")
payload2 = pd.read_json("static_14h12m46s.json", orient="record")
payload = pd.concat([payload1, payload2], ignore_index=True).reset_index()

In [16]:
schema = {
    'relationships': [
         # relation unicité des pdl
         {"fields": ["id_pdc_itinerance", "index"],                    "link" : "coupled" },   
         # relations inter entités
         {"fields": ["id_station_itinerance", "nom_amenageur"],        "link" : "derived" },
         {"fields": ["id_station_itinerance", "siren_amenageur"],      "link" : "derived" },
         {"fields": ["id_station_itinerance", "contact_amenageur"],    "link" : "derived" },
         {"fields": ["id_station_itinerance", "nom_operateur"],        "link" : "derived" },
         {"fields": ["id_station_itinerance", "contact_operateur"],    "link" : "derived" },
         {"fields": ["id_station_itinerance", "telephone_operateur"],  "link" : "derived" },
         {"fields": ["id_station_itinerance", "nom_enseigne"],         "link" : "derived" },
         {"fields": ["id_station_itinerance", "coordonneesXY"],        "link" : "derived" },
         {"fields": ["id_pdc_itinerance", "id_station_itinerance"],    "link" : "derived" },
         # relations intra entité - station
         {"fields": ["id_station_itinerance", "nom_station"],          "link" : "derived" },
         {"fields": ["id_station_itinerance", "implantation_station"], "link" : "derived" },
         {"fields": ["id_station_itinerance", "date_maj"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "num_pdl"],              "link" : "derived" },
         {"fields": ["id_station_itinerance", "id_station_local"],     "link" : "derived" },
         {"fields": ["id_station_itinerance", "nbre_pdc"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "condition_acces"],      "link" : "derived" },
         {"fields": ["id_station_itinerance", "horaires"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "station_deux_roues"],   "link" : "derived" },
         {"fields": ["id_station_itinerance", "raccordement"],         "link" : "derived" },
         {"fields": ["id_station_itinerance", "date_mise_en_service"], "link" : "derived" },
         # relations intra entité - localisation
         {"fields": ["coordonneesXY", "adresse_station"],              "link" : "derived" },
         {"fields": ["coordonneesXY", "code_insee_commune"],           "link" : "derived" }
    ]
}

In [17]:
dic_errors = Cdataset(payload).check_relationship(schema)
dic_errors

{'index - id_pdc_itinerance': (),
 'nom_amenageur - id_station_itinerance': (),
 'siren_amenageur - id_station_itinerance': (),
 'contact_amenageur - id_station_itinerance': (),
 'nom_operateur - id_station_itinerance': (),
 'contact_operateur - id_station_itinerance': (),
 'telephone_operateur - id_station_itinerance': (),
 'nom_enseigne - id_station_itinerance': (),
 'coordonneesXY - id_station_itinerance': (),
 'id_station_itinerance - id_pdc_itinerance': (),
 'nom_station - id_station_itinerance': (),
 'implantation_station - id_station_itinerance': (),
 'date_maj - id_station_itinerance': (),
 'num_pdl - id_station_itinerance': (312,
  313,
  314,
  315,
  316,
  999,
  1000,
  1001,
  1002,
  1003,
  1004,
  1005),
 'id_station_local - id_station_itinerance': (),
 'nbre_pdc - id_station_itinerance': (),
 'condition_acces - id_station_itinerance': (),
 'horaires - id_station_itinerance': (),
 'station_deux_roues - id_station_itinerance': (),
 'raccordement - id_station_itinerance'

In [29]:
payload.loc[dic_errors['num_pdl - id_station_itinerance'], ['num_pdl', 'id_station_itinerance', 'id_pdc_itinerance']]

,num_pdl,id_station_itinerance,id_pdc_itinerance
312,00000000000000,FRHPCPNF080362TOTEMTIERSSA,FRHPCENF080362005
313,00000000000000,FRHPCPNF080362TOTEMTIERSSA,FRHPCENF080362001
314,00000000000000,FRHPCPNF080362TOTEMTIERSSA,FRHPCENF080362002
315,00000000000000,FRHPCPNF080362TOTEMTIERSSA,FRHPCENF080362003
316,00000000000000,FRHPCPNF080362TOTEMTIERSSA,FRHPCENF080362004
999,0,FRHPCPNF080362TOTEMTIERSSA,FRHPCENF080362006
1000,0,FRHPCPNF080362TOTEMTIERSSA,FRHPCENF080362007
1001,0,FRHPCPNF080362TOTEMTIERSSA,FRHPCENF080362008
1002,0,FRHPCPNF080362TOTEMTIERSSA,FRHPCENF080362009
1003,0,FRHPCPNF080362TOTEMTIERSSA,FRHPCENF080362010
